In [1]:
from scipy.special import iv
import numpy as np
import timeit
import viscid
import imp

In [2]:
#If you edit viscid while the kernel is running, run this to reload the package
imp.reload(viscid)

<module 'viscid' from '/Users/zack/Desktop/shaker_docs/faraday/viscid.py'>

In [3]:
args=viscid.args

In [4]:
kx=1
ky=1
omega=1

start=timeit.default_timer()
E = np.zeros((3,3,2*args.Nt+1,2*args.Nt+1,2*args.Nx+1,2*args.Nx+1,2*args.Ny+1,2*args.Ny+1),dtype=np.complex128)
#mode indices. axes appear in the order (l',l,m',m,n',n)
lps=np.arange(-args.Nt,args.Nt+1)[:,np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis]
ls=np.arange(-args.Nt,args.Nt+1)[np.newaxis,:,np.newaxis,np.newaxis,np.newaxis,np.newaxis]
mps=np.arange(-args.Nx,args.Nx+1)[np.newaxis,np.newaxis,:,np.newaxis,np.newaxis,np.newaxis]
ms=np.arange(-args.Nx,args.Nx+1)[np.newaxis,np.newaxis,np.newaxis,:,np.newaxis,np.newaxis]
nps=np.arange(-args.Ny,args.Ny+1)[np.newaxis,np.newaxis,np.newaxis,np.newaxis,:,np.newaxis]
ns=np.arange(-args.Ny,args.Ny+1)[np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis,:]

kappax = kx + args.k1x*ms + args.k2x*ns
kappay = ky + args.k1y*ms + args.k2y*ns
kappa = (kappax**2+kappay**2)**0.5
Omega = 1j*(omega + 2*np.pi*args.freq*ls) + args.mu/args.rho*kappa**2

#Define also here the symmetric sums and differences C,Ctile,S,Stilde above Eq. 31, and then assign the corresponding elements of E with array slicing.
Ctilde = np.exp(-(args.rho/args.mu*Omega)**0.5*args.h0)*iv(ms-mps,(args.rho/args.mu*Omega)**0.5*args.As)*iv(ns-nps,(args.rho/args.mu*Omega)**0.5*args.As) + np.exp((args.rho/args.mu*Omega)**0.5*args.h0)*iv(ms-mps,-(args.rho/args.mu*Omega)**0.5*args.As)*iv(ns-nps,-Omega**0.5*args.As)
E[0,0][(np.arange(E.shape[2]),np.arange(E.shape[3]))] = Ctilde

stop=timeit.default_timer()
print(stop-start)

0.05537885499999984


In [5]:
E=viscid.viscid_mat(1,1,1)
E.shape

(3, 3, 11, 11, 11, 11, 11, 11)